## GPU Demos

### Install and Imports

Do a `pip install` of the [numba](https://numba.pydata.org/) library and check for where the cuda `.so` files are kept.  If a `.so` appears CUDA is likely installed.

In [ ]:
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!pip install pycuda

/usr/local/cuda-10.1/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
     |████████████████████████████████| 1.6MB 6.8MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621013 sha256=241a09f5f7034587ee61fc6c8ca05316251122878c74899420795aebd7b3d474
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=65adc8b244ab7b53c4811143cf0105e5456bead6747b332d889ce25364cf3c70
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


Install libraries

In [ ]:
!nvidia-smi

Wed Nov 25 01:55:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np
from time import time

In [ ]:
ker = """
  __device__ void swap(int & a, int & b){
    int tmp = a;
    a = b;
    b = tmp;
}

__global__ void bitonicSort(int * A, int N){
    extern __shared__ int shared[];
		int tid = threadIdx.x + blockDim.x * blockIdx.x;
    // Copy input to shared mem.
    shared[tid] = A[tid];
    __syncthreads();
    // Parallel bitonic sort.
    for (int k = 2; k <= N; k *= 2){
        // Bitonic merge:
        for (int j = k / 2; j>0; j /= 2){
            int ixj = tid ^ j;
            if (ixj > tid){
                if ((tid & k) == 0){
                    if (shared[tid] > shared[ixj]){
                        swap(shared[tid], shared[ixj]);
                    }
                }
                else{
                    if (shared[tid] < shared[ixj]){
                        swap(shared[tid], shared[ixj]);
                    }
                }
            }
            __syncthreads();
        }
    }
    // Write result.
    A[tid] = shared[tid];
}
"""

In [ ]:
N = 512 #lenght of A
A = np.int32(np.random.randint(1, 20, N)) #random numbers in A
BLOCK_SIZE = N
NUM_BLOCKS = (N + BLOCK_SIZE-1)//BLOCK_SIZE
mod = SourceModule(ker)
bitonicSort = mod.get_function("bitonicSort")
t1 = time()
bitonicSort(drv.InOut(A), np.int32(N), block=(BLOCK_SIZE,1,1), grid=(NUM_BLOCKS,1), shared=4*N)
t2 = time()
print("Execution Time {0}".format(t2 - t1))
print(A)

Execution Time 0.0009696483612060547
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 1